## Lab Assignment Five: Wide and Deep Network

#### Luis Garduno

Dataset: 

--------------------------------------

## 1. Preparation

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 1.1 Loading Data & Adjustments (10%)

In [2]:
%%time
import tensorflow as tf
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.compat.v1.disable_eager_execution()
tf.config.set_soft_device_placement(True)
tf.debugging.set_log_device_placement(True)

a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)


# - Define & prepare my class variables.
# - Use proper variable representations (int, float, one-hot, etc.).
# - Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc.
# - Remove variables that are not needed/useful for the analysis.
# - Describe final dataset used for classification/regression(include a description of any newly formed var's I created).


GPUs:  1
Tensor("MatMul_1:0", shape=(2, 2), dtype=float32)
Wall time: 8.99 ms



### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 1.2 Finding & Creating Cross-Product Features (20%)

In [2]:

# - Identify groups of features in my data that should be combined into cross-product features.



- Provide justification for why these features should be crossed (or why some features shouldn't be crossed). 



### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 1.3 Measuring Algorithm Performance (30%)

- Choose & explain what metric(s) I will use to evaluate your algorithm’s performance.
- I should give a detailed argument for why this (these) metric(s) are appropriate on my data.
- That is, why is the metric appropriate for the task (e.g., in terms of the business case for the task).
- Note: rarely is accuracy the best evaluation metric to use.Think deeply about an appropriate measure of performance.



### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 1.4 Splitting the Dataset (40%)

In [3]:

# - Choose the method I will use for dividing my data into training & testing



- Explain what i'm using & why (i.e Stratified 10-fold cross validation? Shuffle splits?)
- Explain why my chosen method is appropriate or use more than one method as appropriate.
- Argue why my cross validation method is a realistic mirroring of how an algorithm would be used in practice. 



--------------------------

## 2. Modeling


### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.1 Creating Wide & Deep Networks (60%)

In [4]:

# - Create at least 3 combined wide & deep networks to classify your data using Keras.


In [5]:

# - Visualize the performance of the network on the training data & validation data in:
# - the same plot
# - vs.
# - training iterations.
# - Note: use the "history" return parameter that is part of Keras "fit" function to easily access this data.



### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.2 Investigating Generalization Performance (80%)

In [6]:

# - Investigate generalization performance by altering the number of layers in the deep branch of the network.
# - Try at least 2 different number of layers.


In [7]:

# - Use the method of cross validation & evaluation metric I chose at the start of the lab to help select
#   the # of layers that performs superiorly. 



### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.3 Comparing Performaces (90%)

- Use proper statistical method to compare the performance of different models.  

In [8]:

# - Compare the performance of my best wide & deep network to a standard multi-layer perceptron (MLP).


In [9]:

# - For classification tasks, use the receiver operating characteristic & area under the curve.


In [10]:

#- For regression tasks, use Bland-Altman plots & residual variance calculations.



--------------------------------------------------

## 3. t-SNE Dimensionality Reduction

Reference
